In [1]:
import numpy as np
import bluesky as bs
from bluesky import traf, stack, settings
from bluesky.tools.aero import ft, nm, kts
from bluesky.tools.geo import qdrpos, latlondist
from bluesky import navdb

Using Python-based geo functions


In [2]:
TOP = 18000     # Top altitude of TRACON in feet
BOTTOM = 5000   # Bottom altitude of TRACON in feet

In [ ]:
def init_plugin():
    # Create plugin instance
    global approachbs
    approachbs = ApproachBS()
    
    # Configuration parameters
    config = {
        'plugin_name': 'APPROACHBS',
        'plugin_type': 'sim',
        'update_interval': 0.1,  # Update interval in seconds
        'update': approachbs.update,  # Update function
        'preupdate': approachbs.preupdate,  # Preupdate function
        'reset': approachbs.reset  # Reset function
    }
    
    return config


In [9]:
import bluesky as bs
print(bs.scr.__class__)

<class 'bluesky.simulation.screenio.ScreenIO'>


In [10]:
import inspect
print(inspect.getfile(bs.scr.__class__))

c:\myprojects\venv\Lib\site-packages\bluesky\simulation\screenio.py


In [61]:
airport_codes = [
    "KATL", "KLAX", "KORD", "EGLL", "RJTT", "LFPG", "OMDB", "ZSPD", "VHHH", "EDDF",
    "WSSS", "KJFK", "RKSI", "VTBS", "KDFW", "LTBA", "EHAM", "WMKK", "VIDP", "KLAS",
    "KSFO", "YSSY", "KSEA", "CYYZ", "SBGR", "LEMD", "UUEE", "FACT", "EKCH", "LIRF",
    "YMML", "VABB", "RJAA", "KIAH", "EDDM", "KMIA", "LSZH", "OMAA", "KPHX", "LEBL",
    "KEWR", "VTSF", "KPDX", "KDTW", "RCKH", "ZBAA", "RCTP", "KPHL", "KCLT", "SAEZ",
    "MMMX", "LTAC", "LGAV", "BIKF", "EGKK", "RJOO", "KDEN", "LIMC", "TNCM", "CYUL",
    "KMSY", "LOWW", "VECC", "EFHK", "KDCA", "SBGL", "ENGM", "KSTL", "LPPT", "KMCO",
    "WIII", "VTBD", "LFPO", "RJCC", "ZGGG", "ZSHC", "VOBL", "VOTV", "KTPA", "LTFE",
    "LIPZ", "EDDH", "EDLW", "EDDS", "LTAI", "LGTS", "LIRN", "LBSF", "LZIB", "EPWA",
    "LJLJ", "EIDW", "LPPR", "LEAL", "ZSAM", "GCFV", "GCLP", "LPFR", "LGRP", "LGIR",
    "LHBP", "LKPR", "EPKT", "LROP", "ZUUU", "HECA", "LOWI", "HAAB", "LFLL", "LTFJ",
    "PANC", "LFBO", "CYVR", "EGCC"
]
len(airport_codes)

114

In [62]:
len(set(airport_codes))

114

In [3]:
bs.init()

Reading config from C:\Users\zyf36\bluesky\settings.cfg
Reading magnetic variation data
Loading global navigation database...
Reading cache: C:\Users\zyf36\bluesky\cache\navdata.p
Successfully loaded OpenAP performance model
Failed to load BADA performance model
Successfully loaded legacy performance model
Successfully loaded plugin AREA
Successfully loaded plugin DATAFEED


In [63]:
for apt in airport_codes:
    if bs.navdb.getaptidx(apt) == -1:
        print(f"Airport {apt} not found in the database.")

In [4]:
bs.navdb.getaptidx("KOAK")

4102